In [1]:
import pandas as pd
import json
original = pd.read_json("./yelp_dataset/yelp_academic_dataset_business.json", lines=True)
sample = pd.read_csv("./resources/business_sample.csv")
df = pd.read_csv("./resources/business_clean.csv")
top20a = pd.read_csv("./resources/top20attributes.csv", header=None)
top20c = pd.read_csv("./resources/top20Cats.csv", header=None)
original = original[original['business_id'].isin(sample['business_id'])]

In [2]:
state_map = {}
i = 0
for index, row in df.iterrows():
    if not row.state in state_map:
        state_map[row.state] = i
        i += 1

In [3]:
df['state'] = df['state'].map(state_map, na_action='ignore')
clean = df.drop(['attributes', 'categories'], axis=1)

In [4]:
def parseAttributes(d, attribute):
    if type(d) is not dict:
        return -1
    elif attribute in d:
        if d[attribute] == 'True' or d[attribute] == "u'free'" or d[attribute] == 'free':
            return 1
        if d[attribute] == 'False' or d[attribute] == "u'no'" or d[attribute] == 'no' or d[attribute] == 'None':
            return 0
    else:
        for key in d:
            if attribute in key:
                if key[attribute] == 'True' or key[attribute] == "u'free'" or key[attribute] == 'free':
                    return 1
                if key[attribute] == 'False' or key[attribute] == "u'no'" or key[attribute] == 'no'or key[attribute] == 'None':
                    return 0
    return -1

def parseCategories(d, category):
    if category in d:
        return 1
    else:
        return 0

In [5]:
top20list = top20c[0].tolist()[:20]
for i in range(20):
    top20list[i] = top20list[i].split(':')[0]
top20list

for category in top20list:
    original[category] = original['categories'].apply(lambda x: parseCategories(x, category))
for category in top20list:
    clean[category] = original[category].values
    
top20list = top20a[0].tolist()
top20list.remove('RestaurantsAttire')
top20list.remove('NoiseLevel')
top20list.remove('BikeParking')
top20list.remove('Ambience')
top20list.remove('RestaurantsGoodForGroups')
top20list.remove('HasTV')
top20list.remove('RestaurantsPriceRange2')
top20list.remove('GoodForMeal')
top20list.remove('Alcohol')
top20list.remove('BusinessParking')

for attribute in top20list:
    original[attribute] = original['attributes'].apply(lambda x: parseAttributes(x, attribute))
for attribute in top20list:
    clean[attribute] = original[attribute].values

clean['stars > 4'] = clean['stars'].apply(lambda x: x >= 4)
clean['3 < stars < 4'] = clean['stars'].apply(lambda x: 3 <= x < 4)
clean['2 < stars < 3'] = clean['stars'].apply(lambda x: 2 <= x < 3)
clean['1 < stars < 2'] = clean['stars'].apply(lambda x: 1 <= x < 2)
clean = clean.drop(['stars'], axis=1)

clean['hours > 60'] = clean['total_hours'].apply(lambda x: x > 60)
clean['60 > hours > 40'] = clean['total_hours'].apply(lambda x: 60 >= x >= 40)
clean['hours < 40'] = clean['total_hours'].apply(lambda x: x < 40)

clean = clean.drop(['total_hours'], axis=1)

In [6]:
clean

,state,review_count,is_open,Food,American (Traditional),Sandwiches,Nightlife,Bars,Fast Food,American (New),...,Caters,RestaurantsTableService,HappyHour,stars > 4,3 < stars < 4,2 < stars < 3,1 < stars < 2,hours > 60,60 > hours > 40,hours < 40
0,0,116,0,1,1,0,0,0,1,0,...,-1,-1,-1,False,False,True,False,True,False,False
1,1,276,1,1,0,1,0,0,1,1,...,1,-1,-1,True,False,False,False,False,True,False
2,2,18,0,0,0,1,0,0,0,0,...,-1,-1,-1,True,False,False,False,False,False,True
3,3,28,1,1,1,0,0,0,0,0,...,1,1,-1,False,False,False,True,True,False,False
4,0,29,1,1,0,0,0,0,1,0,...,0,0,-1,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,12,32,1,1,0,1,0,0,1,0,...,1,-1,-1,False,False,True,False,True,False,False
196,1,19,1,0,0,0,0,0,0,0,...,1,1,0,False,True,False,False,True,False,False
197,1,29,1,1,0,0,0,0,0,0,...,1,-1,-1,True,False,False,False,False,False,True
198,2,44,1,0,0,0,0,0,0,0,...,-1,-1,-1,False,True,False,False,True,False,False


Generate itemsets of length 1

In [30]:
minSup = 0.2
num_rows = 200.0

itemsets = {}
for col in clean.columns:
    counts = clean[col].value_counts()
    name = counts.name
    counts = counts.to_dict()
    temp = {}
    for c in counts:
        if counts[c] / num_rows >= minSup:
            temp.update({c: counts[c]})
    if temp:
        itemsets.update({name: temp})

In [31]:
itemsets

{'state': {1: 51},
 'is_open': {1: 142, 0: 58},
 'Food': {0: 123, 1: 77},
 'American (Traditional)': {0: 164},
 'Sandwiches': {0: 166},
 'Nightlife': {0: 168},
 'Bars': {0: 163},
 'Fast Food': {0: 175},
 'American (New)': {0: 175},
 'Burgers': {0: 176},
 'Breakfast & Brunch': {0: 178},
 'Pizza': {0: 178},
 'Mexican': {0: 180},
 'Seafood': {0: 180},
 'Chinese': {0: 181},
 'Italian': {0: 184},
 'Delis': {0: 186},
 'Cafes': {0: 187},
 'Coffee & Tea': {0: 189},
 'Chicken Wings': {0: 189},
 'Desserts': {0: 191},
 'Salad': {0: 192},
 'RestaurantsTakeOut': {1: 170},
 'RestaurantsDelivery': {1: 114, 0: 66},
 'BusinessAcceptsCreditCards': {1: 162},
 'OutdoorSeating': {0: 100, 1: 69},
 'RestaurantsReservations': {0: 110, 1: 56},
 'GoodForKids': {1: 136, -1: 43},
 'WiFi': {-1: 104, 0: 48, 1: 48},
 'Caters': {1: 83, -1: 75, 0: 42},
 'RestaurantsTableService': {-1: 131, 1: 44},
 'HappyHour': {-1: 145},
 'stars > 4': {False: 120, True: 80},
 '3 < stars < 4': {False: 114, True: 86},
 '2 < stars < 3':

Generate itemsets of length 2